# Classification Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I **Team, JL2**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### <u>__Predict Overview: EA - Twitter Sentiment Classification 2023-2024__</u>

* To be continued


<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Text Preprocessing</a>

<a href=#five>5. Feature Engineering</a>

<a href=#six>6. Model Training</a>

<a href=#seven>7. Model Perfomance</a>

<a href=#eifght>8. Model Evaluation - Testing set</a>

<a href=#nine>9. Best Model Explanation</a>

# 1. Importing Packages


In [ ]:
# Libraries of loading, manipulation and visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for data preparation and model building

# Setting global constants to ensure notebook results are reproducible

# 2.Loading Data

In [ ]:
# Load the data
df_train = pd.read_csv()
df_test =  pd.read_csv()

# 3. Exploratory Data Analysis (EDA)

In [ ]:
# Shapes, Columns, Data types

# 4. Text Preprocessing

In [ ]:
# Tokenization, Removing stop words etc

# 5. Feature Engineering

## Modeling preparation

In [ ]:
# Feature Selection, Data Split, Data Scaling etc

# 6. Model Training

In [ ]:
# Train Models

# 7. Model Perfomance

In [1]:
# Use various Metrics to evaluate each model's perfomance

# 8. Model Evaluation - Testing set

In [ ]:
# Evaluate the best model on the testing set

# 9. Best Model Explanation

# Submission

In [ ]:
# Create a submission file